<a href="https://colab.research.google.com/github/sakuna47/Fuzzy_logic_CW/blob/Fuzzy_code/Fuzzy_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 8.4 MB/s eta 0:00:00


In [2]:
# Define fuzzy variables
voltage_dev = ctrl.Antecedent(np.arange(0, 21, 1), 'Voltage Deviation (%)')
freq_var = ctrl.Antecedent(np.arange(0, 0.51, 0.01), 'Frequency Variation (Hz)')
load_imb = ctrl.Antecedent(np.arange(0, 31, 1), 'Load Imbalance (%)')
phase_mis = ctrl.Antecedent(np.arange(0, 31, 1), 'Phase Mismatch (degrees)')
severity = ctrl.Consequent(np.arange(0, 101, 1), 'Anomaly Severity')


In [3]:
# Define membership functions
voltage_dev['Low'] = fuzz.trimf(voltage_dev.universe, [0, 2.5, 5])
voltage_dev['Medium'] = fuzz.trimf(voltage_dev.universe, [3, 6.5, 10])
voltage_dev['High'] = fuzz.trimf(voltage_dev.universe, [8, 14, 20])

freq_var['Stable'] = fuzz.trimf(freq_var.universe, [0, 0.1, 0.2])
freq_var['Unstable'] = fuzz.trimf(freq_var.universe, [0.15, 0.325, 0.5])

load_imb['Balanced'] = fuzz.trimf(load_imb.universe, [0, 5, 10])
load_imb['Unbalanced'] = fuzz.trimf(load_imb.universe, [8, 19, 30])

phase_mis['Small'] = fuzz.trimf(phase_mis.universe, [0, 5, 10])
phase_mis['Large'] = fuzz.trimf(phase_mis.universe, [8, 19, 30])

severity['Low'] = fuzz.trimf(severity.universe, [0, 20, 40])
severity['Medium'] = fuzz.trimf(severity.universe, [30, 50, 70])
severity['High'] = fuzz.trimf(severity.universe, [60, 80, 100])